In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import statistics as stat
import seaborn as sns

In [2]:
import pandas as pd
import numpy as np
import transformers
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup, AutoTokenizer, DebertaForSequenceClassification, DebertaModel
import torch
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import torch.nn as nn
from sklearn.metrics import accuracy_score
import pickle as pkl
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, random_split
import string

In [3]:
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [6]:
urw_rec = {
"Blaming the war on others rather than the invader": ["Ukraine is the aggressor", "The West are the aggressors"],
"Discrediting Ukraine": ["Rewriting Ukraine’s history", "Discrediting Ukrainian nation and society", "Discrediting Ukrainian military", "Discrediting Ukrainian government and officials and policies", "Ukraine is a puppet of the West", "Ukraine is a hub for criminal activities", "Ukraine is associated with nazism", "Situation in Ukraine is hopeless"],
"Russia is the Victim": ["The West is russophobic", "Russia actions in Ukraine are only self-defence", "UA is anti-RU extremists"],
"Praise of Russia": ["Praise of Russian military might", "Praise of Russian President Vladimir Putin", "Russia is a guarantor of peace and prosperity", "Russia has international support from a number of countries and people", "Russian invasion has strong national support"],
"Overpraising the West": ["NATO will destroy Russia", "The West belongs in the right side of history", "The West has the strongest international support"],
"Speculating war outcomes": ["Russian army is collapsing", "Russian army will lose all the occupied territories", "Ukrainian army is collapsing"],
"Discrediting the West, Diplomacy": ["The EU is divided", "The West is weak", "The West is overreacting", "The West does not care about Ukraine, only about its interests", "Diplomacy does/will not work", "West is tired of Ukraine"],
"Negative Consequences for the West": ["Sanctions imposed by Western countries will backfire", "The conflict will increase the Ukrainian refugee flows to Europe"],
"Distrust towards Media": ["Western media is an instrument of propaganda", "Ukrainian media cannot be trusted"],
"Amplifying war-related fears": ["By continuing the war we risk WWIII", "Russia will also attack other countries", "There is a real possibility that nuclear weapons will be employed", "NATO should/will directly intervene"],
"Hidden plots by secret schemes of powerful groups": []}

In [7]:
cc_rec = {
"Criticism of climate policies" : ["Climate policies are ineffective", "Climate policies have negative impact on the economy", "Climate policies are only for profit"],
"Criticism of institutions and authorities": ["Criticism of the EU", "Criticism of international entities", "Criticism of national governments", "Criticism of political organizations and figures"],
"Climate change is beneficial": ["CO2 is beneficial", "Temperature increase is beneficial"],
"Downplaying climate change": ["Climate cycles are natural", "Weather suggests the trend is global cooling", "Temperature increase does not have significant impact", "CO2 concentrations are too small to have an impact", "Human activities do not impact climate change", "Ice is not melting", "Sea levels are not rising", "Humans and nature will adapt to the changes"],
"Questioning the measurements and science": ["Methodologies/metrics used are unreliable/faulty", "Data shows no temperature increase", "Greenhouse effect/carbon dioxide do not drive climate change", "Scientific community is unreliable"],
"Criticism of climate movement": ["Climate movement is alarmist", "Climate movement is corrupt", "Ad hominem attacks on key activists"],
"Controversy about green technologies": ["Renewable energy is dangerous", "Renewable energy is unreliable", "Renewable energy is costly", "Nuclear energy is not climate friendly"],
"Hidden plots by secret schemes of powerful groups": ["Blaming global elites", "Climate agenda has hidden motives"],
"Amplifying Climate Fears": ["Earth will be uninhabitable soon", "Amplifying existing fears of global warming", "Doomsday scenarios for humans", "Whatever we do it is already too late"],
"Green policies are geopolitical instruments": ["Climate-related international relations are abusive/exploitative", "Green activities are a form of neo-colonialism"]}

In [8]:
from collections import defaultdict

label2Id = {}
Id2Label = {}
count = 0

dominantLabel2Id = {}
dominantId2Label = {}
dominantCount = 0

for k in urw_rec.keys():

  dominantLabel2Id["URW: "+ k] = dominantCount
  dominantId2Label[dominantCount] = "URW: "+ k
  dominantCount += 1

  urw_rec[k].append("Other")
  for i in urw_rec[k]:
    label = "URW: " + k + ": " + i
    label2Id[label] = count
    Id2Label[count] = label
    count += 1

for k in cc_rec.keys():

  dominantLabel2Id["CC: "+ k] = dominantCount
  dominantId2Label[dominantCount] = "CC: "+ k
  dominantCount += 1

  cc_rec[k].append("Other")
  for i in cc_rec[k]:
    label = "CC: " + k + ": " + i
    label2Id[label] = count
    Id2Label[count] = label
    count += 1

label2Id["Other"] = count
Id2Label[count] = "Other"

dominantLabel2Id["Other"] = dominantCount
dominantId2Label[dominantCount] = "Other"

In [ ]:
len(dominantLabel2Id.keys())

22

In [9]:
enRawFiles = "drive/MyDrive/SemEvalTaskData/target_4_December_release/EN/raw-documents/"
enSubTask2File = "drive/MyDrive/SemEvalTaskData/target_4_December_release/EN/subtask-2-annotations.txt"

hiRawFiles = "drive/MyDrive/SemEvalTaskData/target_4_December_release/HI/raw-documents/"
hiSubTask2File = "drive/MyDrive/SemEvalTaskData/target_4_December_release/HI/subtask-2-annotations.txt"

bgRawFiles = "drive/MyDrive/SemEvalTaskData/target_4_December_release/BG/raw-documents/"
bgSubTask2File = "drive/MyDrive/SemEvalTaskData/target_4_December_release/BG/subtask-2-annotations.txt"

ptRawFiles = "drive/MyDrive/SemEvalTaskData/target_4_December_release/PT/raw-documents/"
ptSubTask2File = "drive/MyDrive/SemEvalTaskData/target_4_December_release/PT/subtask-2-annotations.txt"

ruRawFiles = "drive/MyDrive/SemEvalTaskData/target_4_December_release/RU/raw-documents/"
ruSubTask2File = "drive/MyDrive/SemEvalTaskData/target_4_December_release/RU/subtask-2-annotations.txt"

files_of_interest = "drive/MyDrive/SemEvalTaskData/target_4_December_release/files_of_interest.txt"

In [10]:
files = open(files_of_interest, "r").read().split("\n")

In [11]:
files.pop()

''

In [ ]:
len(files)

1200

In [12]:
from sklearn.metrics import f1_score

def evaluate(model, dataset, label):
  model.eval()

  predicitons = []
  actual_np = []

  with torch.no_grad():
    for i in range(len(dataset)):
      prediction = model(dataset[i]['input_ids'].squeeze(1).to(device), dataset[i]['attention_mask'].squeeze(1).to(device))
      predicitons.append(prediction)
      actual_np.append(dataset[i][label].numpy())

    predcition_tensor = torch.sigmoid(torch.stack(predicitons).squeeze(1))
    predicted_labels_np = torch.where(predcition_tensor > 0.5, 1, 0).cpu().numpy()
    actual_np = np.array(actual_np)

    print("prediction", predicted_labels_np.shape, "actual", actual_np.shape)

    f1_avg_score_micro = f1_score(actual_np, predicted_labels_np, average='micro')
    f1_avg_score_macro = f1_score(actual_np, predicted_labels_np, average='macro')
    f1_score_weighted = f1_score(actual_np, predicted_labels_np, average='weighted')
    f1_avg_score_samples = f1_score(actual_np, predicted_labels_np, average='samples')

    return f1_avg_score_micro, f1_avg_score_macro, f1_score_weighted, f1_avg_score_samples


In [13]:
def evaluateF1Samples(model, dataset, label):
  model.eval()

  predicitons = []
  actual_np = []

  with torch.no_grad():
    for i in range(len(dataset)):
      prediction = model(dataset[i]['input_ids'].squeeze(1).to(device), dataset[i]['attention_mask'].squeeze(1).to(device))
      predicitons.append(prediction)
      actual_np.append(dataset[i][label].numpy())

    predcition_tensor = torch.sigmoid(torch.stack(predicitons).squeeze(1))
    predicted_labels_np = torch.where(predcition_tensor > 0.5, 1, 0).cpu().numpy()
    actual_np = np.array(actual_np)

    print("prediction", predicted_labels_np.shape, "actual", actual_np.shape)

    f1_avg_score_samples = f1_score(actual_np, predicted_labels_np, average='samples')

    return f1_avg_score_samples

In [14]:
class NewsDatasetMultilingual(Dataset):
  def __init__(self):
    self.files = files
    # self.annotation_file = annotations_file
    self.enSubTask2File = enSubTask2File
    self.hiSubTask2File = hiSubTask2File
    self.bgSubTask2File = bgSubTask2File
    self.ptSubTask2File = ptSubTask2File
    self.ruSubTask2File = ruSubTask2File
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

  def __len__(self):
    return len(self.files)

  def __getitem__(self, index: int):
    file_name  = self.files[index]
    text = open(file_name, "r", encoding='utf-8').read()
    text_labels = []
    narr_text_labels = []
    total_labels = [0 for i in range(96)]
    narr_labels = [0 for i in range(22)]

    sub_narrative_df = {}
    narrative_df = {}

    for k in label2Id.keys():
      sub_narrative_df[k] = 0

    for k in dominantLabel2Id.keys():
      narrative_df[k] = 0

    encoded_input = self.tokenizer(text, max_length=512, padding='max_length', truncation=True, return_tensors='pt')

    if "EN" in file_name:
      annotation_file = self.enSubTask2File
    elif "HI" in file_name:
      annotation_file = self.hiSubTask2File
    elif "BG" in file_name:
      annotation_file = self.bgSubTask2File
    elif "PT" in file_name:
      annotation_file = self.ptSubTask2File
    elif "RU" in file_name:
      annotation_file = self.ruSubTask2File

    with open(annotation_file, encoding='utf-8') as annotations:
      for line in annotations:
        splits = line.split("\t")

        if file_name.split("/")[-1] == splits[0]:
          sub_narrative_df['fileName'] = splits[0]
          narrative_df['fileName'] = splits[0]
          text_labels = splits[2].replace("\n", "").split(';')
          narr_text_labels = splits[1].split(';')
          break

    for l in text_labels:
      total_labels[label2Id[l]] = 1
      sub_narrative_df[l] = 1

    for l in narr_text_labels:
      narr_labels[dominantLabel2Id[l]] = 1
      narrative_df[l] = 1


    rec = {}

    rec = {
        **encoded_input,
        "narr_labels": torch.FloatTensor(narr_labels),
        "sub_narr_labels": torch.FloatTensor(total_labels),
        "narr_dic": narrative_df,
        "sub_narr_dic": sub_narrative_df,
        "fileName": file_name.split("/")[-1]
    }

    return rec

In [15]:
total_dataset = NewsDatasetMultilingual()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
model.load_state_dict(torch.load('drive/MyDrive/SemEvalTaskData/model/baselineNarrativeClassifier5Epoch.pth'))
model.eval()

<ipython-input-26-efb1375d8569>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('drive/MyDrive/SemEvalTaskData/model/baselineNarrativeClass

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
evaluate(model, total_dataset, 'narr_labels')

prediction (1200, 22) actual (1200, 22)


(0.7827705397500665, 0.5022660896294767, 0.7367269497059412)

In [ ]:
evaluateF1Samples(model, total_dataset, 'narr_labels')

In [ ]:
total_dataset[0]['input_ids'].squeeze(0).shape

torch.Size([512])

In [ ]:
len(total_dataset)

1200

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self, num_labels):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-uncased', return_dict=True)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)


    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # sequence_output = outputs.pooler_output
        # sequence_output = self.dropout(sequence_output)
        # logits = self.classifier(sequence_output)
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)

        return logits

In [17]:
train_dataset, eval_dataset = random_split(total_dataset, [int(0.9 * len(total_dataset)), len(total_dataset) - int(0.9 * len(total_dataset))])
train_data_loader = DataLoader(train_dataset, shuffle=True, batch_size=5)

In [18]:
len(train_dataset), len(eval_dataset)

(1080, 120)

In [19]:
model = BERTClassifier(22).to(device)

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

In [ ]:
train_loss_history = []
eval_f1_history = []

bceLoss = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

for epoch in range(5):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = bceLoss(outputs, dominantLabels)

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())

        loss.backward()
        optimizer.step()

    eval_f1_micro, eval_f1_macro, eval_f1_weighted, eval_f1_samples = evaluate(model, eval_dataset, 'narr_labels')
    eval_f1_history.append([eval_f1_micro, eval_f1_macro, eval_f1_weighted, eval_f1_samples])
    print("epoch", epoch, "eval f1 micro", eval_f1_micro, "eval f1 macro", eval_f1_macro, "eval f1 weighted", eval_f1_weighted, "eval f1 samples", eval_f1_samples)

print('Finished Training')

epoch 0
epoch 0 step 0 total loss 0.7043805718421936
epoch 0 step 100 total loss 0.22069427371025085
epoch 0 step 200 total loss 0.17955338954925537
prediction (120, 22) actual (120, 22)
epoch 0 eval f1 micro 0.2857142857142857 eval f1 macro 0.11134084062313727 eval f1 weighted 0.23143965066772082 eval f1 samples 0.1640674603174603
epoch 1


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 1 step 0 total loss 0.1670101433992386
epoch 1 step 100 total loss 0.14668022096157074
epoch 1 step 200 total loss 0.1341555416584015
prediction (120, 22) actual (120, 22)
epoch 1 eval f1 micro 0.1864406779661017 eval f1 macro 0.057475246578484594 eval f1 weighted 0.11345803411615017 eval f1 samples 0.1682142857142857
epoch 2


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 2 step 0 total loss 0.16954320669174194
epoch 2 step 100 total loss 0.1722099483013153
epoch 2 step 200 total loss 0.22791768610477448
prediction (120, 22) actual (120, 22)
epoch 2 eval f1 micro 0.49299719887955185 eval f1 macro 0.21275305424544166 eval f1 weighted 0.423625272097076 eval f1 samples 0.41408970658970656
epoch 3


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 3 step 0 total loss 0.16479523479938507
epoch 3 step 100 total loss 0.17327578365802765
epoch 3 step 200 total loss 0.16221070289611816
prediction (120, 22) actual (120, 22)
epoch 3 eval f1 micro 0.4953560371517028 eval f1 macro 0.22059460661630165 eval f1 weighted 0.449459621042246 eval f1 samples 0.4061177248677249
epoch 4


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 4 step 0 total loss 0.15725693106651306
epoch 4 step 100 total loss 0.1527320295572281
epoch 4 step 200 total loss 0.15882724523544312
prediction (120, 22) actual (120, 22)
epoch 4 eval f1 micro 0.5454545454545454 eval f1 macro 0.26116463825002384 eval f1 weighted 0.4789461267227799 eval f1 samples 0.5097222222222223
Finished Training


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
for epoch in range(5, 10):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = bceLoss(outputs, dominantLabels)

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())

        loss.backward()
        optimizer.step()

    eval_f1_micro, eval_f1_macro, eval_f1_weighted, eval_f1_samples = evaluate(model, eval_dataset, 'narr_labels')
    eval_f1_history.append([eval_f1_micro, eval_f1_macro, eval_f1_weighted])
    print("epoch", epoch, "eval f1 micro", eval_f1_micro, "eval f1 macro", eval_f1_macro, "eval f1 weighted", eval_f1_weighted)

print('Finished Training')

epoch 5
epoch 5 step 0 total loss 0.15371790528297424
epoch 5 step 100 total loss 0.0644197165966034
epoch 5 step 200 total loss 0.07369507104158401
prediction (120, 22) actual (120, 22)
epoch 5 eval f1 micro 0.5396825396825397 eval f1 macro 0.31395843577372295 eval f1 weighted 0.5213177998005188
epoch 6


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 6 step 0 total loss 0.06903579831123352
epoch 6 step 100 total loss 0.07078175246715546
epoch 6 step 200 total loss 0.04442048817873001
prediction (120, 22) actual (120, 22)
epoch 6 eval f1 micro 0.5584045584045584 eval f1 macro 0.2515044450562421 eval f1 weighted 0.5106577967043083
epoch 7


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 7 step 0 total loss 0.03472083434462547
epoch 7 step 100 total loss 0.04284067824482918
epoch 7 step 200 total loss 0.02809644490480423
prediction (120, 22) actual (120, 22)
epoch 7 eval f1 micro 0.5414364640883977 eval f1 macro 0.3065898141029177 eval f1 weighted 0.5070232359199611
epoch 8


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 8 step 0 total loss 0.06616881489753723
epoch 8 step 100 total loss 0.040936179459095
epoch 8 step 200 total loss 0.07190601527690887
prediction (120, 22) actual (120, 22)
epoch 8 eval f1 micro 0.5561497326203209 eval f1 macro 0.31766326240821413 eval f1 weighted 0.5281478669346644
epoch 9


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 9 step 0 total loss 0.02218206599354744
epoch 9 step 100 total loss 0.025765877217054367
epoch 9 step 200 total loss 0.049039326608181
prediction (120, 22) actual (120, 22)
epoch 9 eval f1 micro 0.5454545454545454 eval f1 macro 0.294938054204727 eval f1 weighted 0.5188724340038765
Finished Training


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
evaluate(model, total_dataset, 'narr_labels')

prediction (120, 22) actual (120, 22)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.5388888888888889,
 0.2921767068937878,
 0.5120175380867729,
 0.49480158730158735)

In [ ]:
train_f1_micro, train_f1_macro, train_f1_weighted, train_f1_samples = evaluate(model, train_dataset, 'narr_labels')
print("train f1 micro", train_f1_micro, "train f1 macro", train_f1_macro, "train f1 weighted", train_f1_weighted, "train f1 samples", train_f1_samples)

prediction (1080, 22) actual (1080, 22)
train f1 micro 0.9609480204686237 train f1 macro 0.8394315885149376 train f1 weighted 0.9575668711593488 train f1 samples 0.9629129718018606


In [ ]:
evaluateF1Samples(model, eval_dataset, 'narr_labels')

prediction (12, 22) actual (12, 22)


0.6472222222222223

In [ ]:
torch.save(model.state_dict(), "drive/MyDrive/SemEvalTaskData/model/baselineNarrativeClassifier5Epoch.pth")

In [20]:
class GlobalMaxPooling1D(nn.Module):

  def __init__(self, data_format='channels_last'):
      super(GlobalMaxPooling1D, self).__init__()
      self.data_format = data_format
      self.step_axis = 1 if self.data_format == 'channels_last' else 2

  def forward(self, input):
      return torch.max(input, axis=self.step_axis).values

In [21]:
class BERTClassifierPooler(nn.Module):
    def __init__(self, num_labels):
        super(BERTClassifierPooler, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-uncased', return_dict=True)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.pooler = GlobalMaxPooling1D()


    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.pooler(outputs.last_hidden_state)
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        # cls_output = outputs.last_hidden_state[:, 0, :]
        # cls_output = self.dropout(cls_output)
        # logits = self.classifier(cls_output)

        return logits

In [22]:
train_data_loader = DataLoader(train_dataset, shuffle=True, batch_size=5)

In [25]:
subNarrativeClassifier = BERTClassifierPooler(96).to(device)

In [26]:
train_loss_history = []

bceLoss = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.AdamW(subNarrativeClassifier.parameters(), lr=1e-5)

for epoch in range(5):
    subNarrativeClassifier.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        subNarrativeLabels = inputs['sub_narr_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = subNarrativeClassifier(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = bceLoss(outputs, subNarrativeLabels)

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())

        loss.backward()
        optimizer.step()

    eval_f1_micro, eval_f1_macro, eval_f1_weighted, eval_f1_samples = evaluate(subNarrativeClassifier, eval_dataset, 'sub_narr_labels')
    # eval_f1_history.append([eval_f1_micro, eval_f1_macro, eval_f1_weighted, eval_f1_samples])
    print("epoch", epoch, "eval f1 micro", eval_f1_micro, "eval f1 macro", eval_f1_macro, "eval f1 weighted", eval_f1_weighted, "eval f1 samples", eval_f1_samples)

print('Finished Training')

epoch 0
epoch 0 step 0 total loss 0.786078691482544
epoch 0 step 100 total loss 0.22732052206993103
epoch 0 step 200 total loss 0.10312919318675995
prediction (120, 96) actual (120, 96)
epoch 0 eval f1 micro 0.0 eval f1 macro 0.0 eval f1 weighted 0.0 eval f1 samples 0.0
epoch 1


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 1 step 0 total loss 0.10371213406324387
epoch 1 step 100 total loss 0.09876305609941483
epoch 1 step 200 total loss 0.07021377980709076
prediction (120, 96) actual (120, 96)
epoch 1 eval f1 micro 0.0 eval f1 macro 0.0 eval f1 weighted 0.0 eval f1 samples 0.0
epoch 2


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 2 step 0 total loss 0.1103515550494194
epoch 2 step 100 total loss 0.08558846265077591
epoch 2 step 200 total loss 0.09804952889680862
prediction (120, 96) actual (120, 96)
epoch 2 eval f1 micro 0.0 eval f1 macro 0.0 eval f1 weighted 0.0 eval f1 samples 0.0
epoch 3


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 3 step 0 total loss 0.0894518494606018
epoch 3 step 100 total loss 0.1204661875963211
epoch 3 step 200 total loss 0.07204650342464447
prediction (120, 96) actual (120, 96)
epoch 3 eval f1 micro 0.03367003367003367 eval f1 macro 0.005176767676767676 eval f1 weighted 0.022975398585154683 eval f1 samples 0.018922558922558928
epoch 4


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 4 step 0 total loss 0.06894919276237488
epoch 4 step 100 total loss 0.0716594010591507
epoch 4 step 200 total loss 0.05754842981696129
prediction (120, 96) actual (120, 96)
epoch 4 eval f1 micro 0.13166144200626959 eval f1 macro 0.0158650153609831 eval f1 weighted 0.08142391496331026 eval f1 samples 0.10655483405483407
Finished Training


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [27]:
for epoch in range(5, 10):
    subNarrativeClassifier.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        subNarrativeLabels = inputs['sub_narr_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = subNarrativeClassifier(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = bceLoss(outputs, subNarrativeLabels)

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())

        loss.backward()
        optimizer.step()

    eval_f1_micro, eval_f1_macro, eval_f1_weighted, eval_f1_samples = evaluate(subNarrativeClassifier, eval_dataset, 'sub_narr_labels')
    # eval_f1_history.append([eval_f1_micro, eval_f1_macro, eval_f1_weighted, eval_f1_samples])
    print("epoch", epoch, "eval f1 micro", eval_f1_micro, "eval f1 macro", eval_f1_macro, "eval f1 weighted", eval_f1_weighted, "eval f1 samples", eval_f1_samples)

print('Finished Training')

epoch 5
epoch 5 step 0 total loss 0.0908222645521164
epoch 5 step 100 total loss 0.08706250786781311
epoch 5 step 200 total loss 0.03368063271045685
prediction (120, 96) actual (120, 96)
epoch 5 eval f1 micro 0.1588235294117647 eval f1 macro 0.019634856630824375 eval f1 weighted 0.08719793188715297 eval f1 samples 0.13890873015873015
epoch 6


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 6 step 0 total loss 0.09953103214502335
epoch 6 step 100 total loss 0.07174574583768845
epoch 6 step 200 total loss 0.11170713603496552
prediction (120, 96) actual (120, 96)
epoch 6 eval f1 micro 0.11356466876971609 eval f1 macro 0.014101831810165144 eval f1 weighted 0.07228247297933708 eval f1 samples 0.11000721500721501
epoch 7


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 7 step 0 total loss 0.0422753281891346
epoch 7 step 100 total loss 0.043236445635557175
epoch 7 step 200 total loss 0.07011472433805466
prediction (120, 96) actual (120, 96)
epoch 7 eval f1 micro 0.1977401129943503 eval f1 macro 0.03960891856593703 eval f1 weighted 0.1466506650181167 eval f1 samples 0.13010780885780884
epoch 8


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 8 step 0 total loss 0.07135151326656342
epoch 8 step 100 total loss 0.06606895476579666
epoch 8 step 200 total loss 0.044117271900177
prediction (120, 96) actual (120, 96)
epoch 8 eval f1 micro 0.23163841807909605 eval f1 macro 0.051610179943513274 eval f1 weighted 0.17549822483620392 eval f1 samples 0.1734511322011322
epoch 9


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 9 step 0 total loss 0.10481591522693634
epoch 9 step 100 total loss 0.06592901796102524
epoch 9 step 200 total loss 0.04467029124498367
prediction (120, 96) actual (120, 96)
epoch 9 eval f1 micro 0.17647058823529413 eval f1 macro 0.033018208026973805 eval f1 weighted 0.12602726296839117 eval f1 samples 0.12874153624153625
Finished Training


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
outputs.shape

torch.Size([5, 96])

In [ ]:
subNarrativeClassifier.eval()

BERTClassifierPooler(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

In [ ]:
with torch.no_grad():
  prediction = subNarrativeClassifier(total_dataset[50]['input_ids'].squeeze(1).to(device), total_dataset[50]['attention_mask'].squeeze(1).to(device))

In [ ]:
torch.save(subNarrativeClassifier.state_dict(), "drive/MyDrive/SemEvalTaskData/model/baselineSubNarrativeClassifier.pth")

In [ ]:
nn.Sigmoid()(prediction)

tensor([[0.0191, 0.0514, 0.0054, 0.0080, 0.0055, 0.0254, 0.0990, 0.0514, 0.0172,
         0.0227, 0.0309, 0.0268, 0.0257, 0.0247, 0.0109, 0.0265, 0.0352, 0.0223,
         0.0241, 0.0308, 0.0065, 0.0151, 0.0065, 0.0161, 0.0187, 0.0092, 0.0073,
         0.0058, 0.0128, 0.0419, 0.0307, 0.0259, 0.0066, 0.0373, 0.0494, 0.0174,
         0.0942, 0.0199, 0.0058, 0.0318, 0.0175, 0.0054, 0.0091, 0.0513, 0.0317,
         0.0643, 0.0127, 0.0197, 0.0120, 0.0171, 0.0285, 0.0096, 0.0193, 0.0072,
         0.0218, 0.0341, 0.0337, 0.0136, 0.0084, 0.0021, 0.0058, 0.0104, 0.0072,
         0.0033, 0.0088, 0.0052, 0.0035, 0.0061, 0.0021, 0.0046, 0.0072, 0.0037,
         0.0072, 0.0054, 0.0030, 0.0124, 0.0073, 0.0159, 0.0149, 0.0061, 0.0071,
         0.0061, 0.0088, 0.0080, 0.0075, 0.0152, 0.0061, 0.0088, 0.0112, 0.0109,
         0.0061, 0.0144, 0.0050, 0.0136, 0.0029, 0.3476]], device='cuda:0')

In [ ]:
np.argmax(nn.Sigmoid()(prediction)[0].cpu().numpy())

95

In [ ]:
total_dataset[50]['sub_narr_labels']

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])

In [ ]:
np.where(total_dataset[10]['sub_narr_labels'].numpy() == 1)[0]

array([55])

In [ ]:
total_dataset[50]['input_ids'].shape

torch.Size([1, 512])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

In [ ]:
bertModel = BertModel.from_pretrained('bert-base-multilingual-uncased', return_dict=True)

In [ ]:
with torch.no_grad():
  embeddings = bertModel(total_dataset[50]['input_ids'], total_dataset[50]['attention_mask'], output_hidden_states=True)

In [ ]:
narrClassifier = BERTClassifier(22).to(device)

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

In [ ]:
narrClassifier.load_state_dict(torch.load("drive/MyDrive/SemEvalTaskData/model/baselineNarrativeClassifier.pth"))
narrClassifier.eval()

<ipython-input-23-1a7eb0d4d364>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  narrClassifier.load_state_dict(torch.load("drive/MyDrive/SemEvalTaskData/model/baselineNarra

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
predicitons = []

for i in range(len(total_dataset)):
  with torch.no_grad():
    prediction = narrClassifier(total_dataset[i]['input_ids'].squeeze(1).to(device), total_dataset[i]['attention_mask'].squeeze(1).to(device))
    predicitons.append(prediction)


In [ ]:
predicitons[0].shape

torch.Size([1, 22])

In [ ]:
torch.stack(predicitons).squeeze(1).shape

torch.Size([1200, 22])

In [ ]:
predcition_tensor = torch.sigmoid(torch.stack(predicitons).squeeze(1))

In [ ]:
predcition_tensor[0]

tensor([0.0286, 0.0322, 0.0717, 0.0421, 0.0376, 0.0560, 0.0354, 0.0278, 0.0308,
        0.0272, 0.0216, 0.0220, 0.0288, 0.0150, 0.0126, 0.0165, 0.0206, 0.0154,
        0.0152, 0.0260, 0.0082, 0.7358], device='cuda:0')

In [ ]:
predicted_labels_np = torch.where(predcition_tensor > 0.5, 1, 0).cpu().numpy()

In [ ]:
predicted_labels_np.shape

(1200, 22)

In [ ]:
total_dataset[0]['narr_labels']

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1.])

In [ ]:
actual_np = []

for i in range(len(total_dataset)):
  with torch.no_grad():
    actual_np.append(total_dataset[i]['narr_labels'].numpy())

actual_np = np.array(actual_np)

In [ ]:
actual_np.shape

(1200, 22)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_scores = f1_score(actual_np, predicted_labels_np, average=None)

In [ ]:
f1_scores

array([0.046875  , 0.79279279, 0.6440678 , 0.43624161, 0.        ,
       0.        , 0.65957447, 0.        , 0.        , 0.72955975,
       0.        , 0.43037975, 0.62585034, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.94025974,
       0.        , 0.80794702])

In [ ]:
f1_avg_score_micro = f1_score(actual_np, predicted_labels_np, average='micro')
f1_avg_score_macro = f1_score(actual_np, predicted_labels_np, average='macro')
f1_score_weighted = f1_score(actual_np, predicted_labels_np, average='weighted')

In [ ]:
f1_avg_score_micro, f1_avg_score_macro, f1_score_weighted

(0.6181259600614439, 0.27788855744330837, 0.548990554041843)

In [ ]:
class NewsDatasetMultilingual(Dataset):
  def __init__(self):
    self.files = files
    # self.annotation_file = annotations_file
    self.enSubTask2File = enSubTask2File
    self.hiSubTask2File = hiSubTask2File
    self.bgSubTask2File = bgSubTask2File
    self.ptSubTask2File = ptSubTask2File
    self.ruSubTask2File = ruSubTask2File
    self.tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

  def __len__(self):
    return len(self.files)

  def __getitem__(self, index: int):
    file_name  = self.files[index]
    text = open(file_name, "r", encoding='utf-8').read()
    text_labels = []
    narr_text_labels = []
    total_labels = [0 for i in range(96)]
    narr_labels = [0 for i in range(22)]

    sub_narrative_df = {}
    narrative_df = {}

    for k in label2Id.keys():
      sub_narrative_df[k] = 0

    for k in dominantLabel2Id.keys():
      narrative_df[k] = 0

    encoded_input = self.tokenizer(text, max_length=512, padding='max_length', truncation=True, return_tensors='pt')

    if "EN" in file_name:
      annotation_file = self.enSubTask2File
    elif "HI" in file_name:
      annotation_file = self.hiSubTask2File
    elif "BG" in file_name:
      annotation_file = self.bgSubTask2File
    elif "PT" in file_name:
      annotation_file = self.ptSubTask2File
    elif "RU" in file_name:
      annotation_file = self.ruSubTask2File

    with open(annotation_file, encoding='utf-8') as annotations:
      for line in annotations:
        splits = line.split("\t")

        if file_name.split("/")[-1] == splits[0]:
          sub_narrative_df['fileName'] = splits[0]
          narrative_df['fileName'] = splits[0]
          text_labels = splits[2].replace("\n", "").split(';')
          narr_text_labels = splits[1].split(';')
          break

    for l in text_labels:
      total_labels[label2Id[l]] = 1
      sub_narrative_df[l] = 1

    for l in narr_text_labels:
      narr_labels[dominantLabel2Id[l]] = 1
      narrative_df[l] = 1


    rec = {}

    rec = {
        **encoded_input,
        "narr_labels": torch.FloatTensor(narr_labels),
        "sub_narr_labels": torch.FloatTensor(total_labels),
        "narr_dic": narrative_df,
        "sub_narr_dic": sub_narrative_df,
        "fileName": file_name.split("/")[-1]
    }

    return rec

In [ ]:
total_dataset = NewsDatasetMultilingual()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
class DeBERTaClassifier(nn.Module):
    def __init__(self, num_labels):
        super(DeBERTaClassifier, self).__init__()
        self.bert = DebertaModel.from_pretrained('microsoft/deberta-base', return_dict=True)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)


    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # sequence_output = outputs.pooler_output
        # sequence_output = self.dropout(sequence_output)
        # logits = self.classifier(sequence_output)
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)

        return logits

In [ ]:
model = DeBERTaClassifier(22).to(device)

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

In [ ]:
train_dataset, eval_dataset = random_split(total_dataset, [int(0.9 * len(total_dataset)), len(total_dataset) - int(0.9 * len(total_dataset))])
train_data_loader = DataLoader(train_dataset, shuffle=True, batch_size=5)

In [ ]:
train_loss_history = []
eval_f1_history = []

bceLoss = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

for epoch in range(5):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = bceLoss(outputs, dominantLabels)

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())

        loss.backward()
        optimizer.step()

    eval_f1_micro, eval_f1_macro, eval_f1_weighted, eval_f1_samples = evaluate(model, eval_dataset, 'narr_labels')
    eval_f1_history.append([eval_f1_micro, eval_f1_macro, eval_f1_weighted, eval_f1_samples])
    print("epoch", epoch, "eval f1 micro", eval_f1_micro, "eval f1 macro", eval_f1_macro, "eval f1 weighted", eval_f1_weighted, "eval f1 samples", eval_f1_samples)

print('Finished Training')

epoch 0
epoch 0 step 0 total loss 0.6784797310829163
epoch 0 step 100 total loss 0.3301529884338379
epoch 0 step 200 total loss 0.2000536471605301
prediction (120, 22) actual (120, 22)
epoch 0 eval f1 micro 0.06808510638297872 eval f1 macro 0.018648018648018648 eval f1 weighted 0.04308628966163213 eval f1 samples 0.06666666666666667
epoch 1


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 1 step 0 total loss 0.23632517457008362
epoch 1 step 100 total loss 0.20639300346374512
epoch 1 step 200 total loss 0.23894375562667847
prediction (120, 22) actual (120, 22)
epoch 1 eval f1 micro 0.0 eval f1 macro 0.0 eval f1 weighted 0.0 eval f1 samples 0.0
epoch 2


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 2 step 0 total loss 0.20566658675670624
epoch 2 step 100 total loss 0.20445291697978973
epoch 2 step 200 total loss 0.27983081340789795
prediction (120, 22) actual (120, 22)
epoch 2 eval f1 micro 0.21818181818181817 eval f1 macro 0.07720057720057719 eval f1 weighted 0.19232441835181563 eval f1 samples 0.1311111111111111
epoch 3


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 3 step 0 total loss 0.17336751520633698
epoch 3 step 100 total loss 0.13374444842338562
epoch 3 step 200 total loss 0.14533182978630066
prediction (120, 22) actual (120, 22)
epoch 3 eval f1 micro 0.30405405405405406 eval f1 macro 0.13613943483229768 eval f1 weighted 0.2593987861219903 eval f1 samples 0.20878968253968253
epoch 4


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 4 step 0 total loss 0.10798680782318115
epoch 4 step 100 total loss 0.15694059431552887
epoch 4 step 200 total loss 0.25312086939811707
prediction (120, 22) actual (120, 22)
epoch 4 eval f1 micro 0.31438127090301005 eval f1 macro 0.14333192243860993 eval f1 weighted 0.2806979069493719 eval f1 samples 0.1799957912457912
Finished Training


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
for epoch in range(5, 10):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = bceLoss(outputs, dominantLabels)

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())

        loss.backward()
        optimizer.step()

    eval_f1_micro, eval_f1_macro, eval_f1_weighted, eval_f1_samples = evaluate(model, eval_dataset, 'narr_labels')
    eval_f1_history.append([eval_f1_micro, eval_f1_macro, eval_f1_weighted, eval_f1_samples])
    print("epoch", epoch, "eval f1 micro", eval_f1_micro, "eval f1 macro", eval_f1_macro, "eval f1 weighted", eval_f1_weighted, "eval f1 samples", eval_f1_samples)

print('Finished Training')

epoch 5
epoch 5 step 0 total loss 0.1829933524131775
epoch 5 step 100 total loss 0.16444499790668488
epoch 5 step 200 total loss 0.17021027207374573
prediction (120, 22) actual (120, 22)
epoch 5 eval f1 micro 0.29152542372881357 eval f1 macro 0.13987901718164875 eval f1 weighted 0.2565400116319367 eval f1 samples 0.20715608465608465
epoch 6


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 6 step 0 total loss 0.1758723109960556
epoch 6 step 100 total loss 0.17895790934562683
epoch 6 step 200 total loss 0.12984901666641235
prediction (120, 22) actual (120, 22)
epoch 6 eval f1 micro 0.3291139240506329 eval f1 macro 0.14821074863242817 eval f1 weighted 0.2992784995584061 eval f1 samples 0.27403679653679647
epoch 7


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 7 step 0 total loss 0.2280624955892563
epoch 7 step 100 total loss 0.10113441199064255
epoch 7 step 200 total loss 0.12894868850708008
prediction (120, 22) actual (120, 22)
epoch 7 eval f1 micro 0.3375 eval f1 macro 0.16736719582569384 eval f1 weighted 0.30514310651296955 eval f1 samples 0.2698214285714286
epoch 8


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 8 step 0 total loss 0.12375824898481369
epoch 8 step 100 total loss 0.055231187492609024
epoch 8 step 200 total loss 0.09603504091501236
prediction (120, 22) actual (120, 22)
epoch 8 eval f1 micro 0.33766233766233766 eval f1 macro 0.16285970661969246 eval f1 weighted 0.3146988092393185 eval f1 samples 0.26328703703703704
epoch 9


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 9 step 0 total loss 0.17092835903167725
epoch 9 step 100 total loss 0.08195503801107407
epoch 9 step 200 total loss 0.18563134968280792
prediction (120, 22) actual (120, 22)
epoch 9 eval f1 micro 0.3829787234042553 eval f1 macro 0.16970532401605798 eval f1 weighted 0.3354174861609124 eval f1 samples 0.3288095238095238
Finished Training


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
train_f1_micro, train_f1_macro, train_f1_weighted, train_f1_samples = evaluate(model, train_dataset, 'narr_labels')
print("train f1 micro", train_f1_micro, "train f1 macro", train_f1_macro, "train f1 weighted", train_f1_weighted, "train f1 samples", train_f1_samples)

prediction (1080, 22) actual (1080, 22)
train f1 micro 0.7366436203645506 train f1 macro 0.6072887447876355 train f1 weighted 0.7115202219970236 train f1 samples 0.6251945276945277
